# Goal

    Need a dataset with private anonymous ID, survey data, and 7dar
    One row for everyone who received the survey invite
    And 7 day activation rates for everyone who didn't take the survey as wel

2. connecting to wmf repliacs on 3310


In [ ]:
from civilservant.wikipedia.queries.revisions import get_timestamps_within_range

from civilservant.wikipedia.utils import make_cached_df

from civilservant.wikipedia.connections.database import make_wmf_con
import datetime
import os
import pandas as pd
import json

datadir = '/data/project/frwelcome/datasets/survey data/'
survey_f = 'pilot_March_12_2020.csv'
survey_sent_f = 'pilot_experiment_actions_March_12_2020.csv'
acct_map_f = 'pilot_account_mapping.csv'

wmf_con = make_wmf_con()

@make_cached_df('fr-user-edits-pilot-survey')
def fr_user_edits(user_name, start_date):
    end_date = start_date + datetime.timedelta(days=7)
    return get_timestamps_within_range(lang='fr', start_date=start_date, end_date=end_date, user_name=user_name,
                         con=wmf_con)

In [2]:
acct = pd.read_csv(os.path.join(datadir, acct_map_f))

In [3]:
survey = pd.read_csv(os.path.join(datadir, survey_f), skiprows=(1,2)).rename(columns={'ID':'public_anonymous_id'})
survey = survey[pd.notnull(survey['public_anonymous_id'])]
survey = survey.drop_duplicates('public_anonymous_id')

In [4]:
eas = pd.read_csv(os.path.join(datadir, survey_sent_f), sep='\t', parse_dates=['created_dt'])
eas['metadata_json'] = eas['metadata_json'].apply(lambda s: json.loads(s))
eas['public_anonymous_id'] = eas['metadata_json'].apply(lambda j: j['public_anonymous_id'])
# eas['user_name'] = eas['action_object_id']
eas['user_registered_approx_dt'] = eas['created_dt'] - datetime.timedelta(days=85)

In [5]:
eas['public_anonymous_id'].dtype, acct['public_anonymous_id'].dtype

(dtype('O'), dtype('O'))

In [6]:
eas.head(1)

,id,created_dt,experiment_id,action_key_id,action,action_subject_type,action_subject_id,action_object_type,action_object_id,metadata_json,action_platform,removed_dt,public_anonymous_id,user_registered_approx_dt
0,58451,2020-02-24 18:58:04,-12,NaN,talk_page_message,talk_page_message,fr_wiki_welcome_pilot_survey,ThingType.WIKIPEDIA_USER,LUCAS.GR21,"{'lang': 'fr', 'errors': [], 'user_name': 'LUC...",WIKIPEDIA,NaN,c1c34085-15fc-4a42-b6c8-aee2b7fadaf9,2019-12-01 18:58:04


In [7]:
acct.head(1)

,lang,user_name,public_anonymous_id,private_anonymous_id
0,fr,LUCAS.GR21,c1c34085-15fc-4a42-b6c8-aee2b7fadaf9,72e49448-bcfc-3658-a2c8-135e98dcc0fa


In [8]:
eas = pd.merge(eas, acct, on='public_anonymous_id', how='left')

In [9]:
len(eas)

7942

In [31]:
df = pd.merge(survey, eas, how='right', on='public_anonymous_id')

In [32]:
# df = df.set_index('public_anonymous_id')

In [33]:
df.shape, eas.shape, survey.shape

((7942, 41), (7942, 17), (156, 25))

In [34]:
# why did length of df jump up 2
# confirm that the number of users without matching public_anonymous_id is 0

In [35]:
len(df[pd.isnull(df['StartDate'])]), len(eas) - len(survey)

(7786, 7786)

In [36]:
# df = df[:100]

In [37]:
df['timestamps']=df.apply(lambda row: fr_user_edits(row['user_name'], row['user_registered_approx_dt']), axis=1)

In [38]:
def seven_day_revs(timestamps, created_dt):
    seven_days_after_reg = created_dt + datetime.timedelta(days=7)
    return timestamps[timestamps['rev_timestamp']<seven_days_after_reg]

def seven_day_activated(timestamps, created_dt):
    revs = seven_day_revs(timestamps, created_dt)
    return len(revs) > 0

df['seven_day_activated'] = df.apply(lambda row: seven_day_activated(row['timestamps'], row['user_registered_approx_dt']) , axis=1)

In [39]:
df['seven_day_activated'].mean()

0.3684210526315789

In [40]:
out_f = 'frwiki_pilot_survey_behavioural.csv'
skip_cols = ['public_anonymous_id', 'metadata_json', 'timestamp']
out_cols = [c for c in df.columns if c not in skip_cols]
df.to_csv(os.path.join(datadir, out_f), columns=out_cols)

In [28]:
out_df = df[out_cols]